# 深層学習スクラッチ　畳み込みニューラルネットワーク1

このSprintでは1次元の 畳み込み層 を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。


クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

## 実装：問題1〜3
- 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
- 【問題2】1次元畳み込み後の出力サイズの計算
- 【問題3】小さな配列での1次元畳み込み層の実験

In [1]:
import numpy as np

In [2]:
class SimpleConv1d_q1_q3:
    def __init__(self, lr=0.01, sigma=0.01,n_features=784, n_filters=3, n_output=10):
        
        self.lr = lr     # 学習係数
        self.sigma = sigma    # 重みとバイアスの広がり具合
        self.n_features = n_features
        self.n_filters = n_filters    #フィルターのサイズ
        self.n_output = n_output
        
    def fit(self, X, w, b, stride=1,pad=0):
        self.stride = stride
        self.pad = pad
        self.X = X
        n_nodes1 = X.shape[0]
        # 出力サイズを計算
        n_nodes2 = self.calc_out_shape(n_nodes1)
        A1 = np.zeros(n_nodes2)
        A1 = self.forward(A1, n_nodes2, w, b)
        print("A1", A1)
        delta_a = np.array([10, 20])
        db = sum(delta_a)
        print("db", db)
        dw = np.zeros(w.shape[0])
        dw = self.backward(delta_a, n_nodes2, dw)
        print("dw", dw)
        
        dx = np.zeros(self.X.shape[0])

        for i in range(self.X.shape[0]):
            s = 0
            for j in range(n_nodes2):
                if not ((i - j) < 0 or (i - j) > n_nodes2):
                    s += delta_a[j] * w[i - j]
            dx[i] = s
        print("dx", dx)
            
        
    def forward(self, A1, n_nodes2, w, b):
        for i in range(n_nodes2):
            A1[i] = sum(self.X[i : (i + self.n_filters)] * w) + b
        return A1
        
    def backward(self, delta_a, n_nodes2, dw):
        for i in range(dw.shape[0]):
            dw[i] = np.dot(delta_a, self.X[i : (i + n_nodes2)])
        return dw
        
            
    def calc_out_shape(self, n_nodes1):
        # 出力サイズの計算
        n_nodes2 = 1 + (n_nodes1 + 2*self.pad - self.n_filters)/self.stride
        return int(n_nodes2)

In [3]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

sc_conv1 = SimpleConv1d_q1_q3(n_features=4)
sc_conv1.fit(x, w, b)

A1 [35. 50.]
db 30
dw [ 50.  80. 110.]
dx [ 30. 110. 170. 140.]


## 実装：問題4、問題8
- 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
- 【問題8】学習と推定

### 設計：バッチサイズ1

Ich:入力チャンネル、n_features：入力特徴量数、Och:出力チャンネル、Fw:フィルタ幅、Ow:出力幅、n_output：最終出力幅、B:バッチサイズ

【流れ】<br>
X=(Ich, W)、w(フィルタ)=(Och,Ich,Fw)、B=(Och,)<br>
→出力A=(Och,Ow)→活性化関数後Z=(Och,Ow)→フラットにするZ=(B,Och*Ow)→<br>
FC層w=(Och*Ow, n_output)、B=(n_output,)→yp(B,n_output)<br>

逆伝播FC→convのreshapeで戻す動きを念のため確認<br>
reshape(Och, Ow)

In [4]:
A_test = np.array([[1, 2],
                  [3, 4],
                  [5, 6]])
A_test.shape

(3, 2)

In [5]:
A_test = A_test.reshape(1, 6)
A_test

array([[1, 2, 3, 4, 5, 6]])

In [6]:
A_test.reshape(3, 2)

array([[1, 2],
       [3, 4],
       [5, 6]])

### 設計：バッチサイズ複数
Ich:入力チャンネル、n_features：入力特徴量数、Och:出力チャンネル、Fw:フィルタ幅、Ow:出力幅、n_output：最終出力幅、B:バッチサイズ

【流れ】<br>
X=(B,Ich, W)、w(フィルタ)=(B,Och,Ich,Fw)、B=(B,Och)<br>
→出力A=(B,Och,Ow)→活性化関数後Z=(B,Och,Ow)→フラットにするZ=(B,Och*Ow)→<br>
FC層w=(Och*Ow, n_output)、B=(n_output,)→yp(B,n_output)<br>

### 初期値クラス SimpleInitializer

In [26]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2, Fil_w=None):
        """
        重みの初期化
        Parameters
        ----------
        FC層
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        
        conv層(Out_ch、In_ch、Fil_w)
        n_nodes1 : int
          後の層のノード数
        n_nodes2 : int
          前の層のノード数
        Fil_w  :  int
           フィルタ幅
        
        Returns
        ----------
        W :
        """
        if Fil_w == None:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        else:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2, Fil_w)

#         問4確認用
#         W = np.array([[[1., 1., 2.], [2., 1., 1.]],
#                       [[2., 1., 1.], [1., 1., 1.]],
#                       [[1., 1., 1.], [1., 1., 1.]]])
#         W = np.ones((3, 2, 3))
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
#         問4確認用
#         B = np.array([1., 2., 3.]) 
        return B

### 初期値クラス  XavierInitializer

In [8]:
class XavierInitializer:
    def W(self, n_nodes1, n_nodes2, Fil_w=None):
        self.sigma = 1/np.sqrt(n_nodes1)
        if Fil_w == None:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        else:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2, Fil_w)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

### 初期値クラス  HeInitializer

In [9]:
class HeInitializer:
    def W(self, n_nodes1, n_nodes2, Fil_w=None):
        self.sigma = np.sqrt(2/n_nodes1)
        if Fil_w == None:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        else:
            W = self.sigma * np.random.randn(n_nodes1, n_nodes2, Fil_w)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

### Optimizer SGD

In [10]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.w -= self.lr*layer.dW
        layer.b -= self.lr*layer.dB
        
        return layer

### Optimizer AdaGrad

In [11]:
class AdaGrad:
    """
    AdaGrad
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr, hw, hb):
        self.lr = lr
        self.hw = hw
        self.hb = hb
        
    def update(self, layer):
        self.hw = layer.dW**2 + self.hw
        layer.w -= self.lr * np.sqrt(self.hw) * layer.dW
        
        self.hb = layer.dB**2 + self.hb
        layer.b -= self.lr * np.sqrt(self.hb) *  layer.dB
        
        return layer

### 活性化関数 ReLU

In [12]:
class ReLU:
    def __init__(self):
        pass
    
    def forward(self, A):
        self.Z = np.maximum(A, 0)
        return self.Z
    
    def backward(self,dZ):
        dA = np.where(self.Z > 0, dZ, 0)
        return dA

### 活性化関数 Softmax

In [13]:
class Softmax:
    def __init__(self):
        pass
    
    def forward(self, A):
        self.Z = np.exp(A)/np.sum(np.exp(A), axis=1, keepdims=True)
        return self.Z
    
    def backward(self, ty):
        dA = self.Z - ty
        return dA

### 活性化関数 Sigmoid

In [14]:
class Sigmoid:
    def __init__(self):
        pass
    
    def forward(self, A):
        self.Z = 1/(1+np.exp(-A)) 
        return self.Z
    
    def backward(self, dZ):
        dA = dZ * (1 - self.Z) * self.Z
        return dA

### 活性化関数 Tanh

In [15]:
class Tanh:
    def __init__(self):
        pass
    
    def forward(self, A):
        
        self.Z = np.tanh(A)
        return self.Z
    
    def backward(self, dZ):
        dA = dZ * (1 - self.Z**2)    #tanh関数の微分
        return dA

### FC層

In [16]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
        
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w = initializer.W(n_nodes1, n_nodes2)
        self.b = initializer.B(n_nodes2)
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        self.X = X
        A = np.dot(self.X, self.w) + self.b
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dW = np.dot(self.X.T, dA)
        self.dB = np.sum(dA, axis=0)
        self.dZ = np.dot(dA, self.w.T)
        
        # 更新
        self = self.optimizer.update(self)
        
        return self.dZ

### 畳み込み層 SimpleConv1d

In [17]:
class SimpleConv1d:
    """
    Parameters
    ----------
    In_ch : 入力チャネル数
    Out_ch : フィルタ枚数=出力チャネル数
    n_features : 入力特徴量数
    Out_w : 出力サイズ
    Fil_w : フィルタ幅
    stride : ストライド
    pad : パディング
    
    Out_w : 出力サイズ（幅）
    X : 次の形のndarray, shape(In_ch、n_features)
    w : 次の形のndarray, shape(Out_ch、In_ch、Fil_w)
    b : 次の形のndarray, shape（Out_ch）
    """
    def __init__(self, In_ch, Out_ch, n_features, Out_w, Fil_w, stride, pad, initializer, optimizer):
        self.In_ch = In_ch
        self.Out_ch = Out_ch
        self.Out_w = Out_w
        self.n_features = n_features
        self.Fil_w = Fil_w
        self.stride = stride
        self.pad = pad
        self.optimizer = optimizer
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w = initializer.W(self.Out_ch, self.In_ch, self.Fil_w)    # フィルタサイズ
        self.b = initializer.B(self.Out_ch)
    
    def forward(self, X):
        # Xが1次元なら(1, n_features)にreshape
        if X.ndim == 1:
            self.X = X.reshape(-1,self.n_features)
        else:
            self.X = X
        
        A = np.zeros(self.Out_ch * self.Out_w)    #出力A用の入れ物の用意
        A = A.reshape(self.Out_ch, self.Out_w)    #shape(出力チャネル,出力特徴量)

        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            for k in range(self.Out_w):    # 出力サイズ
                A[i][k] = np.sum(np.sum(self.X[ :, k : (k + self.Fil_w)] * self.w[i], axis=1))+ self.b[i]
        return A
    
    def backward(self, dA):
        self.dB = np.sum(dA, axis=1)
        self.dW = np.zeros((self.Out_ch, self.In_ch, self.Fil_w))
        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            s = np.zeros(self.In_ch * self.Fil_w)    # shape(入力チャネル数、フィルタサイズ)
            s = s.reshape(self.In_ch, self.Fil_w)
            for k in range(self.Out_w):    # 出力サイズ
                s += dA[i][k] * self.X[ :, k : (k + self.Fil_w)]
            self.dW[i] = s
        
        self.dX = np.zeros(self.In_ch * self.n_features)
        self.dX = self.dX.reshape(self.In_ch, self.n_features)
        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            for k in range(self.Out_w):    # 出力サイズ
                self.dX[:, 0+self.stride*k : self.Fil_w + self.stride*k] += dA[i][k] * self.w[i]
        # 更新
        self = self.optimizer.update(self)
        return self.dX

In [18]:
class SimpleConv1d_q4:
    """
    Parameters
    ----------
    In_ch : 入力チャネル数
    Out_ch : フィルタ枚数=出力チャネル数
    n_features : 特徴量数
    Fil_w : フィルタ幅
    stride : ストライド
    pad : パディング
    
    Out_w : 出力サイズ（幅）
    X : 次の形のndarray, shape(In_ch、n_features)
    w : 次の形のndarray, shape(Out_ch、In_ch、Fil_w)
    b : 次の形のndarray, shape（Out_ch）
    """
    def __init__(self, In_ch, Out_ch, n_features, Fil_w, stride, pad, initializer, optimizer):
        self.In_ch = In_ch
        self.Out_ch = Out_ch
        self.n_features = n_features
        self.Fil_w = Fil_w
        self.stride = stride
        self.pad = pad
        self.optimizer = optimizer
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w = initializer.W(self.Out_ch, self.In_ch, self.Fil_w)    # フィルタサイズ
        self.b = initializer.B(self.Out_ch)
        
        # 出力サイズを計算
        self.Out_w = self.calc_out_shape()
    
    def calc_out_shape(self):
        # 出力サイズの計算
        return int(1 + (self.n_features + 2*self.pad - self.Fil_w)/self.stride)
    
    def forward(self, X):
        # Xが1次元なら(1, n_features)にreshape
        if X.ndim == 1:
            self.X = X.reshape(-1,self.n_features)
        else:
            self.X = X
        
        A = np.zeros(self.Out_ch * self.Out_w)    #出力A用の入れ物の用意
        A = A.reshape(self.Out_ch, self.Out_w)    #shape(出力チャネル,出力特徴量)
        
        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            for k in range(self.Out_w):    # 出力サイズ
                A[i][k] = np.sum(np.sum(self.X[ :, k : (k + self.Fil_w)] * self.w[i], axis=1))+ self.b[i]
        return A
    
    def backward(self, dA):
        self.dB = np.sum(dA, axis=1)
        self.dW = np.zeros((self.Out_ch, self.In_ch, self.Fil_w))
        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            s = np.zeros(self.In_ch * self.Fil_w)    # shape(入力チャネル数、フィルタサイズ)
            s = s.reshape(self.In_ch, self.Fil_w)
            for k in range(self.Out_w):    # 出力サイズ
                s += dA[i][k] * self.X[ :, k : (k + self.Fil_w)]
            self.dW[i] = s
        
        self.dX = np.zeros(self.In_ch * self.n_features)
        self.dX = self.dX.reshape(self.In_ch, self.n_features)
        for i in range(self.Out_ch):    #フィルタ数＝出力チャネル数
            for k in range(self.Out_w):    # 出力サイズ
                self.dX[:, 0+self.stride*k : self.Fil_w + self.stride*k] += dA[i][k] * self.w[i]
        # 更新
        self = self.optimizer.update(self)
        return self.dX

### Scratch1dCNNClassifier

In [19]:
class Scratch1dCNNClassifier:
    def __init__(self, filter_list, lr=0.01 ,sigma=0.01, n_output=10):
        self.lr = lr     # 学習係数
        self.sigma = sigma    # 重みとバイアスの広がり具合
        self.n_output = n_output
        self.filter_list = filter_list    # フィルタ枚数のリスト
    
    def fit(self, X, y, In_ch, Initializers, activations, Fil_w=3, stride=1, pad=0, opt="SGD", epoch=1):
        self.n_features = X.shape[-1]
        self.loss = np.zeros(epoch)
        self.activation_list = []
        self.FC_list = []
        self.conv_list = []
        self.pad = pad
        self.stride = stride
        self.Fil_w = Fil_w
        
        # conv n_node1:In_ch, n_node2:Out_w / FC  n_node1:入力特徴量,n_node2:出力層サイズ
        n_node1 = self.n_features
        # conv→FC reshape用 flat_Out_ch=フィルタ枚数
        self.flat_Out_ch,self.flat_Out_w = 0, 0
        
        # インスタンス生成---------------------------------------------------
        for i in range(len(Initializers)):
            # 初期値----------------------------------------
            if Initializers[i] == "Simple":
                initializ = SimpleInitializer(self.sigma)
            elif Initializers[i] == "Xavier":
                initializ = XavierInitializer()
            else:
                initializ = HeInitializer()
            # 最適化手法-----------------------------------
            if opt == "SGD":
                optimizer = SGD(self.lr)
            else:
                optimizer = AdaGrad(self.lr, 0, 0)
            # 畳み込み層インスタンス生成-------------------
            if i < len(self.filter_list):
                n_node2 = self.calc_out_shape(n_node1)    # 出力幅の計算
                self.conv_list.append(SimpleConv1d(In_ch, self.filter_list[i], n_node1, n_node2, Fil_w, stride, pad, initializ, optimizer))
            else:
                self.FC_list.append(FC(n_node1, self.n_output, initializ, optimizer))
                
            if i == len(self.filter_list)-1:
                self.flat_Out_ch, self.flat_Out_w = self.filter_list[-1], n_node2    # reshape用に値を取っておく
                n_node1 = self.filter_list[-1]*n_node2    # conv→FCのreshape対応
            else:
                n_node1 = n_node2
            
            # 活性化関数インスタンス生成----------------
            if i < len(self.filter_list):
                if activations[i] == "tanh":
                    self.activation_list.append(Tanh() )
                elif activations[i] == "sigmoid":
                    self.activation_list.append(Sigmoid() )
                else:
                    self.activation_list.append(ReLU() )
            else:
                self.activation_list.append(Softmax() )
        # -----------------------------------------------------------------end
           
        for i in range(epoch):
            
            A = self.conv_list[0].forward(X)    # 畳み込み層 1層目
            Z = self.activation_list[0].forward(A)
            
            for j in range(1, len(self.conv_list) -1):    # 畳み込み層 2層目以降
                A = self.conv_list[j].forward(Z)
                Z = self.activation_list[j].forward(A)
            
            # conv→FCのreshape バッチサイズ=1なので1
            Z = Z.reshape(1, self.flat_Out_ch*self.flat_Out_w)
            
            A = self.FC_list[-1].forward(Z)    # 出力層
            Z = self.activation_list[-1].forward(A)
            
            dA = self.activation_list[-1].backward(y) # 出力層
            dZ = self.FC_list[-1].backward(dA)
            # FC→convのreshape
            dZ = dZ.reshape(self.flat_Out_ch, self.flat_Out_w)
            
            for j in reversed(range(1, len(self.conv_list))):    # 中間層
                dA = self.activation_list[j].backward(dZ)
                dZ = self.conv_list[j].backward(dA)

            dA = self.activation_list[0].backward(dZ)    # 入力層
            dZ = self.conv_list[0].backward(dA)
                
                
            yp = self.predict(X)
            # 損失関数
            self.loss[i] = -np.sum(y * np.log(self.predict_y))/ self.predict_y.shape[1]
            
    def calc_out_shape(self, n_node1):
        # 出力サイズの計算
        return int(1 + (n_node1 + 2*self.pad - self.Fil_w)/self.stride)
    
    def predict(self,X):
        A = self.conv_list[0].forward(X)    # 入力層
        Z = self.activation_list[0].forward(A)
        
        for j in range(1, len(self.conv_list)-1):    # 中間層&出力層
            A = self.conv_list[j].forward(Z)
            Z = self.activation_list[j].forward(A)
            
        # conv→FCのreshape バッチサイズ=1なので1
        Z = Z.reshape(1, self.flat_Out_ch*self.flat_Out_w)
        
        A = self.FC_list[-1].forward(Z)
        Z = self.activation_list[-1].forward(A)
        self.predict_y = Z.copy()
        
        return np.argsort(Z)[:,-1]

### 問4 テストデータ（ホワイトボード）での確認

In [20]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。

In [21]:
initializer_q4 = SimpleInitializer(0.01)
optimizer_q4 = SGD(0.01)

In [22]:
In_ch, Out_ch, n_features, Fil_w, stride, pad =  2, 3, 4, 3, 1, 0
conv_q4 = SimpleConv1d_q4(In_ch, Out_ch, n_features, Fil_w, stride, pad, initializer_q4, optimizer_q4)

In [23]:
conv_q4.forward(x)

array([[21., 29.],
       [18., 25.],
       [18., 24.]])

In [24]:
delta_a =  np.array([[9, 11], [32, 35], [52, 56]])
conv_q4.backward(delta_a)

array([[125., 230., 204., 113.],
       [102., 206., 195., 102.]])

### 問4 テストデータ（diver）での確認

In [39]:
initializer_q4_d = SimpleInitializer(0.01)
optimizer_q4_d = SGD(0.01)

In [40]:
In_ch, Out_ch, n_features, Fil_w, stride, pad =  2, 3, 4, 3, 1, 0
conv_q4_d = SimpleConv1d(In_ch, Out_ch, n_features, Fil_w, stride, pad,
                         initializer_q4_d, optimizer_q4_d)

In [41]:
conv_q4_d.forward(x)

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

### 問題8

In [27]:
np.set_printoptions(precision=8)    # 小数点以下出力桁数
np.set_printoptions(suppress=True)    #指数表示禁止
# 《データセットをダウンロードするコード》
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

from sklearn.preprocessing import OneHotEncoder
# one-hotエンコーディング
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_onehot = enc.fit_transform(y_train[:, np.newaxis])

Using TensorFlow backend.
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [28]:
X_train = X_train[0]
y_train = y_onehot[0]

In [29]:
X_train = X_train.reshape(1, 784)

In [30]:
scCNN_test = Scratch1dCNNClassifier(filter_list=[3])

In [31]:
scCNN_test.fit(X_train, y_train, In_ch=1, Initializers=["Xavier", "Xavier"], activations=["ReLU", "Softmax"],
                opt="AdaGrad", epoch=1)

In [32]:
scCNN_test.loss

array([0.21438927])

In [33]:
scCNN_test.predict(X_train)

array([9])

複数チャンネルデータでも確認

In [34]:
scCNN_test_1 = Scratch1dCNNClassifier(filter_list=[3])

In [35]:
y = np.array([1])

In [36]:
scCNN_test_1.fit(x, y, In_ch=2, Initializers=["Xavier", "Xavier"], activations=["ReLU", "Softmax"],
                opt="AdaGrad", epoch=1)

In [37]:
scCNN_test_1.predict(x)

array([2])